## 0. Load Required Libraries

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import src.util as util

## 1. Import Configuration File

In [7]:
config_data = util.load_config()

## 2. Load Dataset

In [8]:
def load_dataset(config_data: dict) -> pd.DataFrame:
    
    
    # Load every set of data
    clean_data = util.pickle_load(config_data['clean_dataset_path'])

    x_train = util.pickle_load(config_data["train_set_path"][0])
    y_train = util.pickle_load(config_data["train_set_path"][1])

    x_valid = util.pickle_load(config_data["valid_set_path"][0])
    y_valid = util.pickle_load(config_data["valid_set_path"][1])

    x_test = util.pickle_load(config_data["test_set_path"][0])
    y_test = util.pickle_load(config_data["test_set_path"][1])

    # Concatenate x and y each set
    train_set = pd.concat([x_train, y_train], axis = 1)
    valid_set = pd.concat([x_valid, y_valid], axis = 1)
    test_set = pd.concat([x_test, y_test], axis = 1)

    # Return 3 set of data
    return clean_data, train_set, valid_set, test_set

In [9]:
clean_data, train_set, valid_set, test_set = load_dataset(config_data)

## 3. Feature Engineering

### 3.1 Stock Return Data Transform

In [10]:
# as the way to normalize all of data value, its relevant if we change them into return percentage.
# the advantage are: 
# 1. the data value will vary from -0.5 to +0.5. While its possible, its less likely stock change will be up/down more than 50% within 2 days. 
# 2. the stock return is something we want to know anyway therefore its a representative approach in this case

def transform_to_stock_return(dataset, params):
    # define the return for all stock based on the next day of its price change percentage 
    dataset = (dataset.shift(periods=1)-dataset)*100/dataset
    
    #define the target return column name
    target_return_column_name = f"{params['target']} Return D+2"
    
    # add additional column of our targeted stock return
    dataset[target_return_column_name] = dataset[params['target']].shift(periods=-2)

    # handling missing value of shifted targeted column & its reference column
    dataset.dropna(subset=params['target'], inplace=True)
    dataset.dropna(subset=target_return_column_name, inplace=True)

    # handling missing value of the remaining columns
    #dataset.fillna(0, inplace=True)

    return dataset



In [11]:
def remove_outliers(df,n_std):
    for col in df.columns:
        #print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df



In [12]:
train_set_feng = transform_to_stock_return(dataset=train_set, params=config_data)
train_set_feng = remove_outliers(train_set_feng, 3)

In [13]:
val_set_feng = transform_to_stock_return(dataset=valid_set, params=config_data)
val_set_feng = remove_outliers(val_set_feng, 3)

In [14]:
test_set_feng = transform_to_stock_return(dataset=test_set, params=config_data)
test_set_feng = remove_outliers(test_set_feng, 3)

In [15]:
display(train_set_feng.isna().any().sum())
display(util.summary_dataset_describe(dataset=train_set_feng))
display(train_set_feng.shape)

0

,Date,Min,Max
count,580,580.000000,580.000000
mean,2019-08-30 06:44:41.379310336,-0.382604,0.114239
min,2017-01-11 00:00:00,-58.644068,0.000000
25%,2018-06-18 18:00:00,-1.612900,0.000000
50%,2019-09-14 12:00:00,0.000000,0.000000
75%,2020-11-09 06:00:00,0.000000,1.766014
max,2021-08-30 00:00:00,0.000000,52.884615
std,NaN,0.000000,5.482500


(580, 104)

In [16]:
display(val_set_feng.isna().any().sum())
display(util.summary_dataset_describe(dataset=val_set_feng))
display(val_set_feng.shape)

0

,Date,Min,Max
count,146,146.000000,146.000000
mean,2022-01-28 10:11:30.410958848,-0.955964,0.459660
min,2021-09-10 00:00:00,-25.925926,0.000000
25%,2021-11-17 06:00:00,-3.832011,0.000000
50%,2022-01-25 12:00:00,-0.471961,1.452997
75%,2022-03-31 18:00:00,0.000000,5.263162
max,2022-06-27 00:00:00,0.000000,7.526882
std,NaN,0.000000,6.528549


(146, 104)

In [17]:
display(test_set_feng.isna().any().sum())
display(util.summary_dataset_describe(dataset=test_set_feng))
display(test_set_feng.shape)

0

,Date,Min,Max
count,134,134.000000,134.000000
mean,2022-11-18 23:06:16.119403008,-0.732008,0.408796
min,2022-07-05 00:00:00,-25.882353,0.000000
25%,2022-09-20 06:00:00,-3.013468,0.000000
50%,2022-11-22 00:00:00,-0.466203,0.893992
75%,2023-01-24 18:00:00,0.000000,4.182188
max,2023-04-05 00:00:00,0.000000,7.526882
std,NaN,0.000000,6.333179


(134, 104)

## 4. Feature Selection

### 4.1 Filter Correlated Feature

In [18]:
def keep_correlated_features(dataset, params):
    #define the target return column name
    target_return_column_name = f"{params['target']} Return D+2"

    # define the correlated features
    corr_stock = dataset.corrwith(dataset[target_return_column_name], axis=0).nlargest(10)

    # keep correlated features
    dataset = dataset[corr_stock]

    return dataset

In [29]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

def feat_selection(dataset):
    X = dataset.iloc[:,:-1]
    y = dataset.iloc[:,-1]
    model = Lasso(alpha=0.1)
    model.fit(X,y)

    # Get feature coefficients from the Lasso model
    feature_coefficients = model.coef_

    # Create a DataFrame with feature names and their corresponding coefficients
    feature_importances = pd.DataFrame({"feature": X.columns, "coefficient": feature_coefficients})

    # Sort the DataFrame by the absolute value of the coefficients in descending order
    feature_importances = feature_importances.reindex(feature_importances["coefficient"].abs().sort_values(ascending=False).index)

    # Get the top 10 features
    top_10_features = feature_importances.head(10)["feature"].values

    # Print the top 10 features
    print("Top 10 features:", top_10_features)

In [30]:
feat_selection(train_set_feng)
feat_selection(val_set_feng)
feat_selection(test_set_feng)

Top 10 features: ['PTSP.JK' 'AALI.JK' 'BCIC.JK' 'TGKA.JK' 'FAST.JK' 'DUTI.JK' 'BHIT.JK'
 'MEDC.JK' 'LPCK.JK' 'KLBF.JK']
Top 10 features: ['BBCA.JK' 'ASBI.JK' 'BEKS.JK' 'KBLI.JK' 'EMTK.JK' 'BNBR.JK' 'PNLF.JK'
 'JPFA.JK' 'JKON.JK' 'TBIG.JK']
Top 10 features: ['DPNS.JK' 'PTSN.JK' 'PALM.JK' 'HRUM.JK' 'KLBF.JK' 'SMCB.JK' 'BABP.JK'
 'MEGA.JK' 'ERAA.JK' 'IBST.JK']


In [31]:
clean_set_feng = transform_to_stock_return(dataset=clean_data, params=config_data)


In [34]:
def split_data(df):

    train_size = int(len(df)*0.7)
    val_size = int(len(df)*0.85)

    train = df.iloc[:train_size]
    validation = df.iloc[train_size:val_size]
    test = df.iloc[val_size:]

    return train, validation, test

In [43]:
train, val, test = split_data(clean_set_feng)

In [50]:
y_train = train.iloc[:,-1:]
X_train = train.drop(y_train.columns, axis=1)

In [60]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LassoCV

from sklearn.preprocessing import StandardScaler

# Normalize the data
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)


target_col_name = y_train.columns
# Perform feature selection using Lasso with TimeSeriesSplit cross-validation
n_splits = 3
tscv = TimeSeriesSplit(n_splits=n_splits)

# Initialize Lasso with cross-validation
alphas = np.logspace(0, 1, 1000)
lasso_cv = LassoCV(alphas= alphas, cv=tscv)
lasso_cv.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[lasso_cv.coef_ != 0]
display(selected_features)

# 4. Train the final model using the selected features
final_model = Lasso(alpha=lasso_cv.alpha_, random_state=42)
final_model.fit(X_train[selected_features], y_train)

# 5. Evaluate the model on the validation set
y_validation = val[target_col_name]
X_validation = val.drop(target_col_name, axis=1)

y_pred_validation = final_model.predict(X_validation[selected_features])
mse_validation = mean_squared_error(y_validation, y_pred_validation)
print('Mean Squared Error on validation set:', mse_validation)

# 6. Evaluate the final model on the test set
y_test = test[target_col_name]
X_test = test.drop(target_col_name, axis=1)

y_pred_test = final_model.predict(X_test[selected_features])
mse_test = mean_squared_error(y_test, y_pred_test)
print('Mean Squared Error on test set:', mse_test)



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['BCIC.JK'], dtype='object')

Mean Squared Error on validation set: 2.591793762955886
Mean Squared Error on test set: 3.4585836991195076


In [68]:
# Function to create the features for the next few days
def create_features_for_next_days(last_day_data, num_days, selected_features):
    # last_day_data: the last row of your original dataset (as a pandas Series or DataFrame)
    # num_days: number of days you want to forecast
    # selected_features: the list of selected features
    
    # Create a DataFrame containing the features for the next few days
    next_days_data = pd.DataFrame(columns=selected_features, index=pd.date_range(last_day_data.index[-1] + pd.DateOffset(1), periods=num_days, closed='left'))

    # Fill in the feature values based on your feature engineering method
    # For example, if you have lagged features, you can use the last known values to create the new features

    # For this example, let's assume you have lagged features
    # We'll use the last known values to create the features for the next few days
    for feature in selected_features:
        if 'lag' in feature:
            lag = int(feature.split('_')[-1])  # Extract the lag value from the feature name
            next_days_data[feature] = last_day_data['value'].iloc[-lag:].values

    return next_days_data

# Create the features for the next few days
num_days_to_forecast = 5
last_day_data = clean_set_feng.iloc[-1:]
next_days_data = create_features_for_next_days(last_day_data, num_days_to_forecast, selected_features)

# Predict the target value for the next few days
forecast = final_model.predict(next_days_data)

# Print the forecast
forecast_dates = next_days_data.index
for date, value in zip(forecast_dates, forecast):
    print(f"Forecast for {date.strftime('%Y-%m-%d')}: {value}")


/var/folders/fy/zhbysd194v9gxlrksl8f4bp80000gn/T/ipykernel_3191/411603654.py:8: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  next_days_data = pd.DataFrame(columns=selected_features, index=pd.date_range(last_day_data.index[-1] + pd.DateOffset(1), periods=num_days, closed='left'))


ValueError: Input X contains NaN.
Lasso does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

0